In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

In [4]:
training_set = auctions
training_set.sort_values(ascending=True,by='date',inplace=True)

In [5]:
#Separamos el set de entrenamiento en 5 "ventanas" de 3 días cada una.
ventana1 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-18')) & (training_set['date'] < pd.to_datetime('2019-04-21'))]
ventana2 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-19')) & (training_set['date'] < pd.to_datetime('2019-04-22'))]
ventana3 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-20')) & (training_set['date'] < pd.to_datetime('2019-04-23'))]
ventana4 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-21')) & (training_set['date'] < pd.to_datetime('2019-04-24'))]
ventana5 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-22')) & (training_set['date'] < pd.to_datetime('2019-04-25'))]

In [17]:
ventana1['rg_min'] = ventana1.groupby('device_id')['date'].transform(min)
ventana1['rg_max'] = ventana1.groupby('device_id')['date'].transform(max)

train_st = ventana1[['device_id','rg_min','rg_max']].drop_duplicates()
train_st.columns = ['device_id','v1_min','v1_max']

In [18]:
ventana2['rg_min'] = ventana2.groupby('device_id')['date'].transform(min)
ventana2['rg_max'] = ventana2.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana2[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max']

In [19]:
ventana3['rg_min'] = ventana3.groupby('device_id')['date'].transform(min)
ventana3['rg_max'] = ventana3.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana3[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max']

In [20]:
ventana4['rg_min'] = ventana4.groupby('device_id')['date'].transform(min)
ventana4['rg_max'] = ventana4.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana4[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max']

In [21]:
ventana5['rg_min'] = ventana5.groupby('device_id')['date'].transform(min)
ventana5['rg_max'] = ventana5.groupby('device_id')['date'].transform(max)

train_st = pd.merge(train_st,ventana5[['device_id','rg_min','rg_max']].drop_duplicates(),on='device_id',how='inner')
train_st.columns = ['device_id','v1_min','v1_max','v2_min','v2_max','v3_min','v3_max','v4_min','v4_max','v5_min','v5_max']

In [ ]:
train_st['st_v1'] = 0
train_st['st_v2'] = 0
train_st['st_v3'] = 0
train_st['st_v4'] = 0

def f(g):
    g['st_v1'] =  g['v1_max'].iloc[0] - g['v2_min'].iloc[0] 
    g['st_v2'] =  g['v2_max'].iloc[0] - g['v3_min'].iloc[0] 
    g['st_v3'] =  g['v3_max'].iloc[0] - g['v4_min'].iloc[0] 
    g['st_v4'] =  g['v4_max'].iloc[0] - g['v5_min'].iloc[0] 
    return g
train_st = (train_st.groupby('device_id').apply(f))

In [ ]:
train_st = train_st[['device_id','st_v1','st_v2','st_v3','st_v4']]

In [ ]:
no corri mas nada ... esto es prueba emporal hasta definir que esta OK 

In [110]:
df_features = events.groupby('ref_hash').agg({'ref_hash':'size'})
                                           #'event_id':lambda x: x.mode(),
                                           #'application_id':lambda x: x.mode()})

In [111]:
df_features.columns = ['cantidad_de_eventos']#,'evento_mas_frecuente','aplicacion_mas_usada']

In [112]:
Week_Day = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for dia in Week_Day:
    eventos_por_dia = events[events['date'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    eventos_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
    
    df_features = df_features.merge(eventos_por_dia, how='outer', left_index=True, right_index=True)
    
df_features = df_features.fillna(0)

In [113]:
df_features.reset_index(inplace=True)

In [115]:
df_features = pd.merge(df,df_features,left_on='device_id',right_on='ref_hash',how='inner')

In [116]:
df_features.head()

,device_id,ref_hash,cantidad_de_eventos,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday
0,1348993302102753419,1348993302102753419,22,1.0,0.0,0.0,0.0,0.0,0.0,21.0
1,9100483828528005102,9100483828528005102,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3531386881361303675,3531386881361303675,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4785473787968463699,4785473787968463699,5,0.0,0.0,0.0,3.0,0.0,1.0,1.0
4,7106621788075217025,7106621788075217025,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [119]:
del df_features['device_id']
df_features.set_index('ref_hash',inplace=True)

# Modelos de supervivencia multivariables
En el enfoque de Kaplan-Meier utilizado anteriormente, estimamos varias curvas de supervivencia al dividir el conjunto de datos en subgrupos más pequeños según una variable. Si queremos considerar más de 1 o 2 variables, este enfoque rápidamente se vuelve inviable, porque los subgrupos se volverán muy pequeños. En cambio, podemos usar un modelo lineal, el modelo de riesgo proporcional de Cox, para estimar el impacto que tiene cada variable en la supervivencia.

Sin embargo, primero debemos convertir las variables categóricas en el conjunto de datos en valores numéricos.

In [120]:
from sksurv.preprocessing import OneHotEncoder

df_features_numeric = OneHotEncoder().fit_transform(df_features)

In [121]:
df_features_numeric.head()

,cantidad_de_eventos,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday
ref_hash,,,,,,,,
1348993302102753419,22,1.0,0.0,0.0,0.0,0.0,0.0,21.0
9100483828528005102,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3531386881361303675,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4785473787968463699,5,0.0,0.0,0.0,3.0,0.0,1.0,1.0
7106621788075217025,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0


Los modelos de supervivencia en scikit-survival siguen las mismas reglas que los estimadores en scikit-learn, es decir, tienen un método de ajuste que espera una matriz de datos y un conjunto estructurado de tiempos de supervivencia e indicadores de eventos binarios.

In [128]:
ventana1_st.loc[ventana1_st['st']<259200,'data_cs'] = True
ventana1_st.loc[ventana1_st['st']>=259200,'data_cs'] = False

In [129]:
test_ventana1 = ventana1_st[['data_cs','st']].to_records(index=False) 

,device_id,st,data_cs
0,1814167382111032,104847.548340,True
1,5197805394267564,259200.000000,False
2,6380906313550611,99860.790413,True
3,7164788605058735,125034.961021,True
4,10151602175762247,59730.605603,True


In [130]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(df_features_numeric,test)

CoxPHSurvivalAnalysis(alpha=0, n_iter=100, tol=1e-09, verbose=0)

In [131]:
pd.Series(estimator.coef_, index=df_features_numeric.columns)

cantidad_de_eventos          -0.006096
cantidad_eventos_Monday       0.005727
cantidad_eventos_Tuesday      0.007544
cantidad_eventos_Wednesday    0.006556
cantidad_eventos_Thursday     0.006151
cantidad_eventos_Friday       0.005475
cantidad_eventos_Saturday     0.006208
cantidad_eventos_Sunday       0.007411
dtype: float64

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [132]:
prediction = estimator.predict(df_features_numeric)

In [133]:
estimator.score(df_features_numeric, test)

0.5080844975042663

# KNN

In [138]:
set_training = pd.merge(ventana1_st,df_features.reset_index(), how='inner', left_on='device_id',right_on='ref_hash')

In [141]:
del set_training['device_id']

In [143]:
columns = list(set_training.columns)
columns.remove('st')
columns.remove('data_cs')

In [144]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(set_training.loc[:, columns],
                                                    set_training['st'],
                                                    test_size=0.2,
                                                    random_state=43)

In [147]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [148]:
neigh.fit(X_train,Y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [152]:
predicted = neigh.predict(X_test) #Predict the target for the provided data

In [156]:
neigh.score(X_test, Y_test)

-0.3494937581195072